In [27]:
from langsmith import Client 

import os 
import openai 

from qdrant_client import QdrantClient 
from langchain_openai import ChatOpenAI  
from langchain_openai import OpenAIEmbeddings 

from ragas.llms import LangchainLLMWrapper 
from ragas.embeddings import LangchainEmbeddingsWrapper 

### Download an example reference data point from LangSmith

In [4]:
client = Client()

In [5]:
dataset = client.read_dataset(dataset_name="rag-evaluation-dataset")

In [6]:
dataset

Dataset(name='rag-evaluation-dataset', description='Dataset for evaluating RAG pipeline', data_type=<DataType.kv: 'kv'>, id=UUID('7314766e-ca06-4404-8814-934b83990229'), created_at=datetime.datetime(2025, 10, 10, 2, 34, 29, 737011, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 10, 10, 2, 34, 29, 737011, tzinfo=datetime.timezone.utc), example_count=37, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None, metadata={'runtime': {'sdk': 'langsmith-py', 'library': 'langsmith', 'runtime': 'python', 'platform': 'Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.31', 'sdk_version': '0.4.33', 'runtime_version': '3.12.11', 'langchain_version': None, 'py_implementation': 'CPython', 'langchain_core_version': '0.3.78'}})

In [8]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].inputs

{'question': 'Do you sell power banks or portable battery chargers?'}

In [9]:
list(client.list_examples(dataset_id=dataset.id, limit=10))[0].outputs

{'ground_truth': 'We do not currently have any power banks or portable battery chargers listed in stock.',
 'reference_context_ids': [],
 'reference_description': []}

In [25]:
list(client.list_examples(dataset_id=dataset.id, limit=15))[11].inputs

{'question': 'Which products would be suitable if I want noise-canceling wireless earbuds?'}

In [ ]:
list(client.list_examples(dataset_id=dataset.id, limit=15))[11].outputs

{'ground_truth': 'Options include TELSOR Wireless Earbuds (B0C6K1GQCF) and S23-vine Wireless Earbuds (B0B9FTVL58), both featuring noise-canceling tech and long battery life. RUSAM GA33 Game Earbuds (B09WCFC5D9) also offer noise cancellation and are suitable for gaming and music.',
 'reference_context_ids': ['B0C6K1GQCF', 'B0B9FTVL58', 'B09WCFC5D9'],
 'reference_description': ["TELSOR Wireless Earbuds for iPhone, Bluetooth Headphones Touch Control Stereo Sound Bluetooth Earbuds with Noise Cancelling Mic for Calls, 30H Playtime, IPX7 Waterproof Earbuds for Android, Black ♬【Bluetooth】Pair instantly with an uninterrupted and stable transmission with Bluetooth 5.1. AVRCP, HCP, HSP, and A2DP profiles are supported. The wireless earbuds are compatible with most Bluetooth enabled iPhones, Andriods, smart TVs, computers, etc. Each wireless earbuds will pair with each other when they are removed from the charging case. From here, enable Bluetooth on your chosen device and pair with the headphone

In [26]:
reference_input = list(client.list_examples(dataset_id=dataset.id, limit=15))[11].inputs
reference_output = list(client.list_examples(dataset_id=dataset.id, limit=15))[11].outputs 

### Rag Pipeline

In [ ]:
# def get_embedding(text, model="text-embedding-3-small"):
#     response = openai.embeddings.create(
#         input = text,
#         model = model,
#     )

#     current_run = get_current_run_tree()
#     if current_run: 
#         current_run.metadata["usage_metadata"] = {
#             "input_tokens": response.usage.prompt_tokens,
#             "total_tokens": response.usage.total_tokens,
#         }

#     return response.data[0].embedding

# @traceable(
#     name="retrieve_data",
#     run_type="retriever"
# )
# def retrieve_data(query, qdrant_client, k=5): 
#     query_embedding = get_embedding(query)  
#     results = qdrant_client.query_points(
#         collection_name="Amazon-items-collection-00",  
#         query=query_embedding, 
#         limit=k,  
#     ) 
#     retrieved_context_ids = []
#     retrieved_context = [] 
#     similarity_scores = [] 
    
#     for result in results.points: 
#         retrieved_context_ids.append(result.payload["parent_asin"]) 
#         retrieved_context.append(result.payload["description"]) 
#         similarity_scores.append(result.score) 

#     return {
#         "retrieved_context_ids": retrieved_context_ids, 
#         "retrieved_context": retrieved_context, 
#         "similarity_scores": similarity_scores, 
#     }

# @traceable(
#     name="format_retrieved_context",
#     run_type="prompt"
# )
# def process_context(context):
#     formatted_context = "" 

#     for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):  
#         formatted_context += f"- {id}: {chunk}\n"  

#     return formatted_context

# @traceable(
#     name="build_prompt",
#     run_type="prompt"
# )
# def build_prompt(preprocessed_context, question): 

#     prompt = f""" 
#     You are a shopping assistant that can answer qeustions about the products in stock. 

#     You will be given a question and a list of context. 

#     Instructions: 
#     - You need to answer the question based on the provided context only. 
#     - Never use word context and refer to it as the available products. 

#     Context: 
#     {preprocessed_context} 

#     Question: 
#     {question} 
#     """ 
#     return prompt 

# @traceable(
#     name="generate_answer",
#     run_type="llm",
#     metadata={"ls_provider": "openai", "ls_model_name": "gpt-4.1-mini"}
# )
# def generate_answer(prompt):  
#     response = openai.chat.completions.create(
#         model="gpt-4o-mini", 
#         messages=[{"role": "system", "content": prompt}], 
#         temperature=0.5, 
#     ) 

#     current_run = get_current_run_tree()
#     if current_run: 
#         current_run.metadata["usage_metadata"] = {
#             "input_tokens": response.usage.prompt_tokens,
#             "output_tokens": response.usage.completion_tokens,
#             "total_tokens": response.usage.total_tokens,
#         }

#     return response.choices[0].message.content

# @traceable(
#     name="rag_pipeline"
# )
# def rag_pipeline(question, top_k=5): 
#     qdrant_client = QdrantClient(url="http://qdrant:6333") 

#     retrieved_context = retrieve_data(question, qdrant_client, top_k) 
#     preprocessed_context = process_context(retrieved_context)
#     prompt = build_prompt(preprocessed_context, question)
#     answer = generate_answer(prompt)

#     final_result = {
#         "answer": answer, 
#         "question": question,  
#         "retrieved_context_ids": retrieved_context["retrieved_context_ids"],   
#         "retrieved_context": retrieved_context["retrieved_context"],   
#         "similarity_scores": retrieved_context["similarity_scores"],   
#     }
#     return final_result